This notebook contains the scaling of the baseline model, e.g. normalization and standardization with the ImageDataGenerator.

The first part will explain the different steps of scaling, afterwards both scaling methods will be used on the baseline model.

__Overview scaling with the ImageDataGenerator__

The following steps are done with the data split into training and validation data.

In [ ]:
# Create an ImageDataGenerator and input the chosen scaling choices (also augmentation is possible)
# for Normalization: (rescale = 1.0/255.0)
# for Standardization (this includes Centering): (featurewise_center = True, featurewise_std_normalization = True)
datagen= ImageDataGenerator(rescale = 1.0/255.0) #this would be normalization

# if needed (depends on scaling method), calculate for the whole training data set the statistics using the .fit() function. Later on this can be applied to test and validation data set.
datagen.fit(X_train)

print('Train min=%.3f, max=%.3f' % (X_train.min(), X_train.max()))

# A neural network model can be fitted with the data generator by using .flow() . It retrieves an iterator which returns batches of data and passes it to the fit_generator() function.

# creating the iterator, choose the wanted batch size
train_iterator = datagen_normalization.flow(X_train, y_train, batch_size = 64)


# Optional: confirm that the iterators and the scaling work
batchX, batchy = train_iterator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(),
                                              batchX.max()))

The fit() is applied to the model and the train_iterator is chosen. It will take the scaled data and feed it into the model.
A hyperparameter here is the number of epochs and the steps per epoch, which can be chosen accordingly.

For larger datasets the function fit_generator() can be used. It will divide the data into batches and scale in-time during training while feeding the batches to the model. For the fit() function the whole data would be stored in RAM, which is not always possible for larger or more complex data sets.

For later plotting purposes the output is additionally stored in history.

In [ ]:
# Fitting the model
history = model.fit(train_iterator, steps_per_epoch=len(train_iterator), epochs=50)

The last step is to evaluated the model with the validation data.
First the validation data part from the read in data (see subset = "validation") is chosen and prepared in the same way as the train data. 
A validation iterator is created and finally the model is evaluated.

In [ ]:
# fitted model is validated

val_data = tf.keras.utils.image_dataset_from_directory(
    data_directory,
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="validation"
)

# Convert labels to NumPy arrays
X_val = np.concatenate([x for x, _ in val_data], axis=0)
y_val = np.concatenate([y for _, y in val_data], axis=0)

# Convert labels to one-hot encoded format
y_val = to_categorical(y_val, num_classes=4)

# an iterator is created from the validation data.
val_iterator = datagen.flow(X_val, y_val, batch_size = 64) 

# Evaluate the model on the validation data
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

__In the following are some preparations for the baseline model__

In [ ]:
!pip install scikit-learn

In [ ]:
!pip install opencv-python

In [ ]:
%pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical
import os
from sklearn.model_selection import train_test_split


# converts a class vector (integers) to binary class matrix 
from keras.utils import to_categorical
# The ImageDataGenerator itself
from keras.preprocessing.image import ImageDataGenerator


__Reading in the data with image_dataset_from_directory__

This splits the data into train and validation data (later used) and also resizes the images to the given image_size.

In [ ]:
# Define the directory path containing the images
data_directory = "/Users/linn/Desktop/original_dataset"
batch_size = 32
image_size = (200,200)
data = tf.keras.utils.image_dataset_from_directory(
    data_directory,
    batch_size=batch_size,
    image_size = image_size,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="training")


In [ ]:
# Print the class names
class_names = data.class_names
print("Class names:", class_names)


In [ ]:
# Convert labels to NumPy arrays
X_train = np.concatenate([x for x, _ in data], axis=0)
y_train = np.concatenate([y for _, y in data], axis=0)


In [ ]:
# Convert labels to one-hot encoded format
y_train = to_categorical(y_train, num_classes=4)

Setting up the CNN model

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the baseline model architecture
model_normalization = tf.keras.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')
])

# Compile the model
model_normalization.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model_normalization.summary()

First a normalization is tried:

In [ ]:
# Create an ImageDataGenerator and input the chosen scaling choices 
datagen_normalization = ImageDataGenerator(rescale = 1.0/255.0)

# the data set is scaled
datagen_normalization.fit(X_train)

# A neural network model can be fitted with the data generator by using .flow() . It retrieves an iterator which returns batches of data and passes it to the fit_generator() function.

# creating the iterator, choose the wanted batch size
train_iterator = datagen_normalization.flow(X_train, y_train, batch_size = 64)

# confirming scaling
batchX, batchy = train_iterator.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(),
                                              batchX.max()))


In [ ]:
# fit the model
history = model_normalization.fit(train_iterator, steps_per_epoch=len(train_iterator), epochs=20)

In [ ]:
# fitted model is validated

val_data = tf.keras.utils.image_dataset_from_directory(
    data_directory,
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="validation"
)

# Convert labels to NumPy arrays
X_val = np.concatenate([x for x, _ in val_data], axis=0)
y_val = np.concatenate([y for _, y in val_data], axis=0)

# Convert labels to one-hot encoded format
y_val = to_categorical(y_val, num_classes=4)

val_iterator = datagen_normalization.flow(X_val, y_val, batch_size = 64) 

# Evaluate the model on the validation data
val_loss, val_accuracy = model_normalization.evaluate(X_val, y_val)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)


The model accuracy unfortunately goes down in validation to 25%.

The following is another type of validation, it achieves the same accuracy (25%)

In [ ]:
# evaluate model
_, acc = model_normalization.evaluate_generator(val_iterator, steps=len(val_iterator), verbose=0)
print('Test Accuracy: %.3f' % (acc * 100))

In [ ]:
# plotting the loss and accuracy
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history.history['accuracy'])
#plt.plot(history_val[val_accuracy])
plt.title('model accuracy normalization')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
#plt.plot(val_)
plt.title('model loss normalization')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

Let's see how the standardization is doing:

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the baseline model architecture
model_standardization = tf.keras.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(4, activation='softmax')
])

# Compile the model
model_standardization.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print the model summary
model_standardization.summary()

In [ ]:
# Create an ImageDataGenerator and input the chosen scaling choices (
# for Standardization (this includes Centering): (featurewise_center = True, featurewise_std_normalization = True)
datagen_standardization = ImageDataGenerator(featurewise_center = True, featurewise_std_normalization = True)

# fit scaling on the train data
datagen_standardization.fit(X_train)

# creating the iterator, choose the wanted batch size
train_iterator_std = datagen_standardization.flow(X_train, y_train, batch_size = 64)


# confirm that the iterator works
batchX, batchy = train_iterator_std.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(),
                                              batchX.max()))


In [ ]:
# Fitting the model
history_std = model_standardization.fit(train_iterator_std, steps_per_epoch=len(train_iterator), epochs=20)

In [ ]:
# plotting the loss and accuracy
import matplotlib.pyplot as plt

# summarize history for accuracy
plt.plot(history_std.history['accuracy'])
plt.title('model accuracy standardization')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history_std.history['loss'])
plt.title('model loss standardization')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Train'], loc='upper left')
plt.show()

In [ ]:
# fitted model is validated

val_data = tf.keras.utils.image_dataset_from_directory(
    data_directory,
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
    seed=42,
    validation_split=0.2,
    subset="validation"
)

# Convert labels to NumPy arrays
X_val = np.concatenate([x for x, _ in val_data], axis=0)
y_val = np.concatenate([y for _, y in val_data], axis=0)

# Convert labels to one-hot encoded format
y_val = to_categorical(y_val, num_classes=4)

val_iterator_std = datagen_standardization.flow(X_val, y_val, batch_size = 64) 

# Evaluate the model on the validation data
val_loss, val_accuracy = model_standardization.evaluate(X_val, y_val)
print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_accuracy)

The model accuracy with standardization is 28%.

The model is also evaluated with another approach with the same outcome.

In [ ]:
# evaluate model
_, acc = model_standardization.evaluate_generator(val_iterator_std, steps=len(val_iterator_std), verbose=0)
print('Test Accuracy: %.3f' % (acc * 100))

Standardization and Normalization do not improve the model accuracy.
Maybe another opmtimization tool is needed as well for the scaling to improve the accuracy.
Baseline model: 33%
Scaled with normalization: 25%
Scaled with standardization: 28%